In [22]:
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry
import matplotlib.pyplot as plt

In [ ]:
#import climate data  -- Openmeteo

In [20]:
#function for retrieving dataframes for the wished locations
def get_weather_data(latitude, longitude, start_date, end_date, elevation):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": [latitude],
        "longitude": [longitude],
        "start_date": start_date,
        "end_date": end_date,
        "hourly": ["temperature_2m", "precipitation", "cloud_cover", "wind_speed_10m"],
        "elevation": [elevation]
    }

    # Get weather data for the specified location
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]  # Assuming only one location is requested

    # Process hourly data
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
    hourly_cloud_cover = hourly.Variables(2).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(3).ValuesAsNumpy()

    # Create DataFrame to store hourly weather data
    hourly_data = {
        "date": pd.date_range(
            start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
            end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = hourly.Interval()),
            inclusive = "left"
        )
    }
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["cloud_cover"] = hourly_cloud_cover
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

    hourly_dataframe = pd.DataFrame(data = hourly_data)
    return hourly_dataframe



In [39]:
# Example usage:
latitude = 40.3
longitude = -8.2
start_date = "2022-01-01"
end_date = "2024-01-01"
elevation = 124
file = 'Aguieira_weather_data.csv'

# Call the function with the specified parameters
weather_data = get_weather_data(latitude, longitude, start_date, end_date, elevation)
weather_data.to_csv(file)